In [4]:
import API_flightapi as API
import pandas as pd
import json
from pprint import pprint

import DB_Mongo

io = open("requetes/flightapi_schedule.json","r")
data = json.load(io)

#pprint(data)

legs = pd.DataFrame(data['legs'])
price = pd.DataFrame(data['fares'])
airport = pd.DataFrame(data['airports'])
airline = pd.DataFrame(data['airlines'])

def billet_id_split(df):
    temp = df.str.split(pat=':', n=- 1, expand=True)
    result = temp[1] + '-' + temp[2]
    result = result.str.split(pat='-0', n=-1, expand=True)[0]
    
    return result
    

legs['join_id'] = billet_id_split(legs['id'])
price['join_id'] = price['tripId'].str.split(pat=':', n=- 1, expand=True)[1]


#legs = legs.rename({'departureAirportCode':'code', 'airlineCodes':'code'}, axis = 1)

Jointure = legs.merge(right = price, on='join_id', how = 'inner')
#Jointure = Jointure.merge(right = airport, on='join_id', how = 'inner')
#Jointure = Jointure.merge(right = airline, on='join_id', how = 'inner')

price = Jointure['price']

price = price.tolist()
        
for i, index in enumerate(price):
    price[i] = price[i]['totalAmount']

Jointure['price'] = price


NewTab = Jointure[['departureTime','arrivalTime','duration','departureAirportCode','arrivalAirportCode','airlineCodes','price']]

(NewTab.head(10)).to_json(r'requetes/test_jointure.json')

db = DB_Mongo.DB_Mongo()
db.insert_vol(legs.to_dict(orient='list'))

db.insert_event(NewTab.to_dict(orient='list'))

#print(list(db.vol.find(projection={'legs':1, '_id':0})))

[{}, {}, {}, {}]
